In [1]:
from model import TextClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

tqdm.pandas()
encoder = LabelEncoder()

2022-12-07 20:04:53.889980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 20:04:56.128931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:04:56.129389: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:04:56.129403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Good to go!


In [3]:
class MyDataset(Dataset):
    def __init__(self):
        
        self.df = None

        self.embeddings = None
        self.labels = None

        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None

    def load_embeddings(self, file_path):
        # Use if you've already generated spacy embeddings
        self.df = pd.read_json(file_path)
        # Convert the embeddings to nd array
        self.df['vector'] = self.df['vector'].apply(np.array)
        # Separate the embeddings and labels as series
        self.embeddings = self.df['vector']
        self.labels = self.df['num_cat']
    
    def get_train(self):
        return zip(self.y_train, self.x_train)
    
    def get_test(self):
        return zip(self.y_test, self.x_test)
    
    def split_test_train(self):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.embeddings, self.labels, test_size=0.20)

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [4]:
huffPo = MyDataset()
huffPo.load_embeddings('/common/users/shared/cs543_fall22_group3/huffpo/spacy_vectors.json')
huffPo.split_test_train()

: 

: 

In [6]:
category_mapping = {c: huffPo.df.loc[huffPo.df['num_cat'] == c, 'category'].iloc[0] for c in range(42)}

In [7]:
label_pipeline = lambda x: int(x)

In [8]:
# Function to create batches of our data
def collate_batch(batch):
    label_list, embedding_list = [], []
    
    for (_label, _embedding) in batch:
        label_list.append(label_pipeline(_label))
        embedding = torch.tensor(_embedding, dtype=torch.float32)
        embedding_list.append(embedding)

    label_list = torch.tensor(label_list, dtype=torch.int64)
    embedding_list = torch.stack(embedding_list)
    
    return label_list.to(device), embedding_list.to(device)

train_iter = huffPo.get_train()
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [ ]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    acc, pred = [], []

    with torch.no_grad():
        for idx, (label, vector) in enumerate(dataloader):
            predicted_label = model(vector)
            loss = criterion(predicted_label, label)

            acc.extend(label.cpu().detach().numpy())
            pred.extend(predicted_label.argmax(1).cpu().detach().numpy())

            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    
    return acc, pred

In [14]:
# Hyperparameters
EPOCHS = 30  # epoch
LR = 0.1  # learning rate
BATCH_SIZE = 64 # batch size for training

model = TextClassificationModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = huffPo.get_train()
test_iter = huffPo.get_test()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [16]:
evaluate(test_dataloader, model, criterion)

tensor([ 6, 30, 40, 39, 17, 24, 10, 23, 38, 38,  1, 38, 40, 24, 41, 14, 30, 38,
        22, 38, 18, 17, 38,  0, 38,  3, 24,  3, 24, 24, 22, 10, 25, 38, 25, 10,
        38,  9, 13, 12, 38, 38, 24, 24, 34, 36, 13,  9, 24, 18, 24, 13, 18, 38,
        10,  2, 14, 33, 24,  3, 34, 24, 38, 40], device='cuda:0') tensor([[-1.9065,  0.8094,  0.1730,  ..., -0.2851, -1.9694,  0.2497],
        [-1.3786, -0.3287,  0.1276,  ..., -1.0468, -1.5532,  0.0176],
        [-1.9799,  0.6536, -0.8242,  ..., -2.4431,  0.5682,  1.0874],
        ...,
        [-1.6231,  1.2030, -2.4291,  ...,  1.2952, -3.0438,  1.7988],
        [-1.6795, -0.1766, -2.1319,  ..., -2.1389, -1.9456, -0.4947],
        [-0.3346,  1.5925, -1.2161,  ..., -0.4849, -3.8439,  0.7384]],
       device='cuda:0')
tensor([24, 26,  3, 14, 32,  1, 29, 11,  3, 24,  5, 40, 38, 38, 13, 33, 38, 38,
        10, 30, 38,  5, 22,  8, 34, 24, 40, 36, 25, 24, 29, 16, 30, 24, 17, 25,
        38, 10, 22, 29, 10, 18, 38, 12, 22, 13, 40, 33,  4, 25, 34, 28, 27, 

0.0011215577721567318

In [ ]:
torch.save(model.state_dict(), '/common/users/shared/cs543_fall22_group3/models/class_model.pt')

In [ ]:
model.eval()

In [ ]:
my_tensor = torch.stack().unsqueeze(1)
my_tensor